In [0]:
"""
Created on Tue Oct 15 00:14:08 2019

@author: anil
"""

import pandas as pd
import numpy as np
     
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from nltk.tokenize import word_tokenize 

import nltk
nltk.download("popular")
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print(accelerator)

from google.colab import drive
drive.mount('/content/drive')



## Data Reading

In [0]:

"""
Test mode 

"""
olid_training=pd.read_csv("/content/drive/My Drive/OFFENSEVAL20-DATA/offenseval-tr-training-v1.tsv",sep="\t")
X_train_FULL=olid_training[["id","tweet","subtask_a"]] 
Y_train_FULL=olid_training["subtask_a"]
X_test_FULL=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/offenseval-tr-testset-v1.tsv',sep='\t',encoding='utf8',quoting=csv.QUOTE_NONE)
Y_TEST=pd.read_csv('/content/drive/My Drive/TURKISH-DATA/turkish-goldlabels.tsv',sep=',',encoding='utf8',quoting=csv.QUOTE_NONE,header=None)
Y_TRAIN_ENCODED_FULL=[1 if i ==  'OFF' else 0 for i in Y_train_FULL]
Y_TEST_ENCODED_FULL = [1 if i ==  'OFF' else 0 for i in Y_TEST[1]]

## Pre-processing

In [0]:
"""
	 Converting all text to lowercase 
	 Removing all "\#" symbols
	 Removing all punctuation
	 Removing @user tokens
"""
X_train_FULL.tweet = X_train_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_train_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_train_FULL["tweet_initial"]=filtered_tweets


X_test_FULL.tweet = X_test_FULL.tweet.str.lower()

filtered_tweets=[]
for tweet in X_test_FULL["tweet"]:
    tweet_tokens = word_tokenize(tweet) 

    filtered_sentence = [w for w in tweet_tokens if (( w!='user' and w!='@' and w!=',' and w!= "'" and w!='.'and w!='#' and w!='?'))] 
      
    filtered_tweets.append(filtered_sentence)

X_test_FULL["tweet_initial"]=filtered_tweets




#FOR TEST

z=[]
for tweet in X_train_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_train_FULL["tweet_initial_nontoken"]=z



#FOR TEST

z=[]
for tweet in X_test_FULL["tweet_initial"]:
    d=" ".join(tweet)
    z.append(d)
X_test_FULL["tweet_initial_nontoken"]=z



In [0]:
"""
Importing Keras dependencies / Write Custom Evaluation Metrices
"""

from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from tensorflow.keras.layers import Embedding
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
def recall_m(true_Y, pred_Y):
        TP = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        possible_pos = K.sum(K.round(K.clip(true_Y, 0, 1)))
        rec = TP / (possible_pos + K.epsilon())
        return rec

def precision_m(true_Y, pred_Y):
        true_positives = K.sum(K.round(K.clip(true_Y * pred_Y, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(pred_Y, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision


    

def f1_m(true_Y, pred_Y):
    pres = precision_m(true_Y, pred_Y)
    rec = recall_m(true_Y, pred_Y)
    return 2*((pres*rec)/(pres+rec+K.epsilon()))

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


## Word2vec /

In [0]:
fname= "/content/drive/My Drive/Twitter/Word2Vec/w2v_model_word.vec"
import gensim
from gensim.models import Word2Vec
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(fname)  # you can load this saved keyedvectors model later



## Tokenizing / creating vocabulary and wordindex using keras functinalities


In [0]:
"""
We will use word indexes as look-up table during embedding layer.
"""
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=80202)  #the maximum number of words to keep, based on word frequency. Only the most common num_words-1 words will be kept.
tokenizer.fit_on_texts(X_train_FULL["tweet_initial_nontoken"])
X_train_initial = tokenizer.texts_to_sequences(X_train_FULL["tweet_initial_nontoken"])
X_test_initial = tokenizer.texts_to_sequences(X_test_FULL["tweet_initial_nontoken"])
vocab_size_initial = len(tokenizer.word_index) + 1 
wordIndex_initial=tokenizer.word_index # it is  index
from keras.preprocessing.sequence import pad_sequences
max_len = 50

"""
Padding

"""
X_train_initial = pad_sequences(X_train_initial, padding='post', maxlen=max_len)
X_test_initial = pad_sequences(X_test_initial, padding='post', maxlen=max_len)




Custom Embedding Layer

In [0]:
 """
Custom Embedding Layer with Word2vec weights
 """

def createEmbeddingLayer(wordIndex,not_static):
  a=[]
  embedding_dim=300
  vocabulary_size=len(wordIndex)+1
  embedding_matrix = np.zeros((vocabulary_size, embedding_dim))
  missed=0
  for word, i in wordIndex.items():
    
          
      try:
          embedding_vector = word_vectors[word] # or fast text
          embedding_matrix[i] = embedding_vector
            

      except KeyError: # If word is not found in the word2vec vocabulary , assign random weights
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),embedding_dim)
        missed+=1
        a.append(word)

  print('missed_words :' , missed)

  custom_embedding_layer = Embedding(vocabulary_size,
                                embedding_dim,
                                weights=[embedding_matrix],
                                trainable=not_static # Controls the updating weights )
  return custom_embedding_layer

  

## BiLSTM / Attention

In [0]:
"""
BiLSTM-Attention Network

Benefited from https://github.com/ShawnyXiao/TextClassification-Keras#5-textattbirnn

Implementation of https://arxiv.org/abs/1512.08756
"""


from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer

class Attention_feed_forward(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
     
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0

        super(Attention_feed_forward, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight( shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='w',
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='b',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        e = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))  
        if self.bias:
            e += self.b
        e = K.tanh(e)

        a = K.exp(e)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
  
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)

        c = K.sum(a * x, axis=1)
        return c

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.features_dim

    

In [0]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Embedding, Dense, Bidirectional,LSTM

def sigmoid(x): 
    return 1.0/(1 + np.exp(-x))


class Attention_BiLSTM(object):
    def __init__(self, maxlen, embedding_dims,
                 class_num=1,
                 last_activation='sigmoid'):
        self.maxlen = maxlen
        self.embedding_dims = embedding_dims
        self.class_num = class_num
        self.last_activation = last_activation

    def get_model(self):
        input = Input((self.maxlen,))
        embedding = createRandomTextEmbeddingLayer(wordIndex_initial,True)(input) #change
        x = Bidirectional(LSTM(128, return_sequences=True))(embedding)  # LSTM or GRU
        x = Attention_feed_forward(self.maxlen)(x)

        output = Dense(self.class_num, activation=self.last_activation)(x)
        model = Model(inputs=input, outputs=output)
        return model

#Benefited from https://github.com/ShawnyXiao/TextClassification-Keras#5-textattbirnn
 

In [0]:
def train(X_train_initial,X_test_initial,y_train,y_test) : 
  bilstm_attention_model= Attention_BiLSTM(50, 100).get_model()

  early_stopping = [EarlyStopping(monitor='val_loss',min_delta=0,restore_best_weights=True, patience=10,verbose=1, mode='auto')]
  bilstm_attention_model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['acc',f1_m,precision_m, recall_m])
  bilstm_attention_model.summary()

  bilstm_attention_model.fit(X_train_initial, y_train,
            batch_size=16,
            epochs=30,
            callbacks=early_stopping,
            validation_split=0.1)

  loss, accuracy, f1_score, precision, recall = bilstm_attention_model.evaluate(X_train_initial, y_train, verbose=1)
  print("Attention- BiRNN Training Loss: {:.4f}".format(loss))
  print("Attention- BiRNN Accuracy: {:.4f}".format(accuracy))
  print("Attention- BiRNN f1 score: {:.4f}".format(f1_score))
  print("Attention- BiRNN Precision: {:.4f}".format(precision))
  print("Attention- BiRNN Recall: {:.4f}".format(recall))

  loss, accuracy, f1_score, precision, recall = bilstm_attention_model.evaluate(X_test_initial, y_test, verbose=1)
  print("Attention- BiRNN Test Loss: {:.4f}".format(loss))
  print("Attention- BiRNN Test Accuracy: {:.4f}".format(accuracy))
  print("Attention- BiRNN Test f1 score: {:.4f}".format(f1_score))
  print("Attention- BiRNN Test Precision: {:.4f}".format(precision))
  print("Attention- BiRNN Test Recall: {:.4f}".format(recall))
 
  probs = bilstm_attention_model.predict(X_test_initial, verbose=1)
  print('lenght of probs : ' ,len(probs))
  predicted_classes=[0 if i < 0.5 else 1 for i in probs]
 

  print(classification_report(y_test, predicted_classes,digits=3))
  return (predicted_classes,probs)


prediction_attention,probs_attention=train(np.array(X_train_initial),np.array(X_test_initial),np.array(Y_TRAIN_ENCODED_FULL),np.array(Y_TEST_ENCODED_FULL))

